In [57]:
import numpy as np
with np.load("notMNIST.npz") as data :
    Data, Target = data ["images"], data["labels"]
    posClass = 2
    negClass = 9
    dataIndx = (Target==posClass) + (Target==negClass)
    Data = Data[dataIndx]/255.
    Target = Target[dataIndx].reshape(-1, 1)
    Target[Target==posClass] = 1
    Target[Target==negClass] = 0
    np.random.seed(521)
    randIndx = np.arange(len(Data))
    np.random.shuffle(randIndx)
    Data, Target = Data[randIndx], Target[randIndx]
    trainData, trainTarget = Data[:3500], Target[:3500]
    validData, validTarget = Data[3500:3600], Target[3500:3600]
    testData, testTarget = Data[3600:], Target[3600:]

In [48]:
import tensorflow as tf
def SGD_loss(decay_scale, X, y):
    #initializer
    weight_init = tf.contrib.layers.xavier_initializer(uniform=True)
    const_init = tf.constant_initializer(0.01)
    
    #weight bias
    W1 = tf.get_variable("W1",shape=[X.shape[1],1],initializer=weight_init, dtype=tf.float32)
    b1  = tf.get_variable("b1",shape=[1,1],initializer=const_init, dtype=tf.float32)#broadcast
    #hidden layer
    y_ = tf.add(tf.matmul(X, W1), b1)
    print(y_)
    print(y)
    #loss
    weight_decay = decay_scale/2*tf.matmul(tf.transpose(W1),(W1))
    loss = tf.nn.l2_loss(y_-y)/tf.cast(tf.shape(X)[0],tf.float32)+weight_decay
    return loss

In [64]:
def train(trainData_rs,trainTarget,learnning_rate,batch_size=500,steps=20000):
    trainning_data = tf.constant(trainData_rs, dtype=tf.float32)
    trainning_label = tf.constant(trainTarget, dtype=tf.float32)
    #feed_dict = {trainning_data:trainData_rs, trainning_label:trainTarget}
    x, y = tf.train.slice_input_producer([trainning_data, trainning_label], num_epochs=None)
    #y = tf.cast(y, tf.int32)
    X, Y = tf.train.batch([x, y], batch_size=batch_size)
    
    loss = SGD_loss(0, X, Y)
    optimizer = tf.train.GradientDescentOptimizer(learnning_rate)
    train_op = optimizer.minimize(loss)
    
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    
    sess = tf.Session()
    sess.run(init_op)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
  
    for step in range(steps):
        _, loss_val = sess.run([train_op, loss])#, feed_dict = feed_dict)
        print(loss_val)
        


In [ ]:
BATCH_SIZE = 500
LEARNING_RATES = (0.005, 0.001, 0.0001)
tf.reset_default_graph()

trainData_rs = trainData.reshape([trainData.shape[0], -1])

train(trainData_rs, trainTarget, LEARNING_RATES[0], BATCH_SIZE, 20000)

Tensor("Add:0", shape=(500, 1), dtype=float32)
Tensor("batch:1", shape=(500, 1), dtype=float32)
[[ 0.15172505]]
[[ 0.14191397]]
[[ 0.13202807]]
[[ 0.10600234]]
[[ 0.09945117]]
[[ 0.10451929]]
[[ 0.09040158]]
[[ 0.09435542]]
[[ 0.08819123]]
[[ 0.09018835]]
[[ 0.08572998]]
[[ 0.08003375]]
[[ 0.08658347]]
[[ 0.08522022]]
[[ 0.07742165]]
[[ 0.06876659]]
[[ 0.07008567]]
[[ 0.08670682]]
[[ 0.0756306]]
[[ 0.08296949]]
[[ 0.08435944]]
[[ 0.07560327]]
[[ 0.07322716]]
[[ 0.07167231]]
[[ 0.0628761]]
[[ 0.07432202]]
[[ 0.07113001]]
[[ 0.07810631]]
[[ 0.06841958]]
[[ 0.08054961]]
[[ 0.06976958]]
[[ 0.05684701]]
[[ 0.06653211]]
[[ 0.07503658]]
[[ 0.0612391]]
[[ 0.0722118]]
[[ 0.06922452]]
[[ 0.06539717]]
[[ 0.05514498]]
[[ 0.0585895]]
[[ 0.06717995]]
[[ 0.06890625]]
[[ 0.06094157]]
[[ 0.0582727]]
[[ 0.06869609]]
[[ 0.06116276]]
[[ 0.05945832]]
[[ 0.06464287]]
[[ 0.06543108]]
[[ 0.06826896]]
[[ 0.05406414]]
[[ 0.0472662]]
[[ 0.06987569]]
[[ 0.05969537]]
[[ 0.07118849]]
[[ 0.05235647]]
[[ 0.05783918]]

[[ 0.02775896]]
[[ 0.03189341]]
[[ 0.0273076]]
[[ 0.03447192]]
[[ 0.03475585]]
[[ 0.02931062]]
[[ 0.02765719]]
[[ 0.02786688]]
[[ 0.03085968]]
[[ 0.03452221]]
[[ 0.02843464]]
[[ 0.02740066]]
[[ 0.03187037]]
[[ 0.03127865]]
[[ 0.02826463]]
[[ 0.02749958]]
[[ 0.03384716]]
[[ 0.02862716]]
[[ 0.03054192]]
[[ 0.03127935]]
[[ 0.03150499]]
[[ 0.03221038]]
[[ 0.02816889]]
[[ 0.02946456]]
[[ 0.03391248]]
[[ 0.02699439]]
[[ 0.0279914]]
[[ 0.03175607]]
[[ 0.03230323]]
[[ 0.03264602]]
[[ 0.03383688]]
[[ 0.02656118]]
[[ 0.03037618]]
[[ 0.02687798]]
[[ 0.02708202]]
[[ 0.03076836]]
[[ 0.02972986]]
[[ 0.03137426]]
[[ 0.02957932]]
[[ 0.02848168]]
[[ 0.02947522]]
[[ 0.02948663]]
[[ 0.02682955]]
[[ 0.03220244]]
[[ 0.02528604]]
[[ 0.03480186]]
[[ 0.03041898]]
[[ 0.02925213]]
[[ 0.02912805]]
[[ 0.02953932]]
[[ 0.0289696]]
[[ 0.02701085]]
[[ 0.03134533]]
[[ 0.02743718]]
[[ 0.0340024]]
[[ 0.02952642]]
[[ 0.0300891]]
[[ 0.03319656]]
[[ 0.02930791]]
[[ 0.02853476]]
[[ 0.02572331]]
[[ 0.0293279]]
[[ 0.0302066]]

[[ 0.02500481]]
[[ 0.02341752]]
[[ 0.02439516]]
[[ 0.02391816]]
[[ 0.02790933]]
[[ 0.01814486]]
[[ 0.02391816]]
[[ 0.02929655]]
[[ 0.02747902]]
[[ 0.02710404]]
[[ 0.02301668]]
[[ 0.0247451]]
[[ 0.0270741]]
[[ 0.02862077]]
[[ 0.0262552]]
[[ 0.02246198]]
[[ 0.02611408]]
[[ 0.01977185]]
[[ 0.02325281]]
[[ 0.0258643]]
[[ 0.02605009]]
[[ 0.0242497]]
[[ 0.02209982]]
[[ 0.02432773]]
[[ 0.02986172]]
[[ 0.02055518]]
[[ 0.0235125]]
[[ 0.0214918]]
[[ 0.02496169]]
[[ 0.02839862]]
[[ 0.02244684]]
[[ 0.02323895]]
[[ 0.02879355]]
[[ 0.02562593]]
[[ 0.02791212]]
[[ 0.0290763]]
[[ 0.01996973]]
[[ 0.02602497]]
[[ 0.02181841]]
[[ 0.02216402]]
[[ 0.02574718]]
[[ 0.01984147]]
[[ 0.02701531]]
[[ 0.02375246]]
[[ 0.02863242]]
[[ 0.02222538]]
[[ 0.02479198]]
[[ 0.02062771]]
[[ 0.02468027]]
[[ 0.02380318]]
[[ 0.02536076]]
[[ 0.0251139]]
[[ 0.02813631]]
[[ 0.02429131]]
[[ 0.02780629]]
[[ 0.0238287]]
[[ 0.02167899]]
[[ 0.0241105]]
[[ 0.02641528]]
[[ 0.02143967]]
[[ 0.02643098]]
[[ 0.02541733]]
[[ 0.02037626]]
[[ 

[[ 0.0194525]]
[[ 0.01856813]]
[[ 0.02433478]]
[[ 0.01900429]]
[[ 0.02311733]]
[[ 0.02202258]]
[[ 0.02205896]]
[[ 0.0245243]]
[[ 0.02153243]]
[[ 0.02290627]]
[[ 0.01999286]]
[[ 0.02231202]]
[[ 0.02199648]]
[[ 0.02226542]]
[[ 0.02161902]]
[[ 0.02130518]]
[[ 0.02578299]]
[[ 0.02265585]]
[[ 0.02303417]]
[[ 0.02313386]]
[[ 0.02210844]]
[[ 0.02407559]]
[[ 0.01896777]]
[[ 0.02116419]]
[[ 0.01980755]]
[[ 0.02186658]]
[[ 0.02275877]]
[[ 0.02337467]]
[[ 0.02605274]]
[[ 0.01979908]]
[[ 0.02106743]]
[[ 0.02505002]]
[[ 0.02332287]]
[[ 0.02200636]]
[[ 0.02210027]]
[[ 0.02157265]]
[[ 0.01889962]]
[[ 0.02183891]]
[[ 0.0224291]]
[[ 0.02170761]]
[[ 0.02186707]]
[[ 0.02072082]]
[[ 0.02326311]]
[[ 0.02370792]]
[[ 0.02070977]]
[[ 0.02185048]]
[[ 0.01985614]]
[[ 0.02133991]]
[[ 0.02174441]]
[[ 0.02022492]]
[[ 0.02770161]]
[[ 0.02148409]]
[[ 0.02258838]]
[[ 0.02173792]]
[[ 0.02118988]]
[[ 0.0208445]]
[[ 0.02308458]]
[[ 0.02214448]]
[[ 0.0223298]]
[[ 0.01944503]]
[[ 0.0219788]]
[[ 0.01792444]]
[[ 0.02271975]